# 🎬 Instagram Reels Generator

Generate engaging Instagram reels with AI-powered scripts and automated video creation.
This notebook creates short videos with text overlays, voiceovers, and background music.

In [ ]:
# Install required packages
!pip install transformers torch moviepy gtts pillow numpy opencv-python gradio requests textwrap3 pydub

In [ ]:
import os
import json
import re
import random
import textwrap
from datetime import datetime
from pathlib import Path

# AI and NLP
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Video and Audio
from moviepy.editor import *
from gtts import gTTS
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from pydub import AudioSegment

# Web interface
import gradio as gr

# Utilities
import requests
import io
import base64

## 1. Script Generation Engine

In [ ]:
class ReelsScriptGenerator:
    def __init__(self, model_name="microsoft/DialoGPT-medium"):
        self.model_name = model_name
        self.initialize_model()
        
        # Reel templates for different content types
        self.templates = {
            "educational": {
                "hook": "Did you know that {topic}?",
                "structure": ["Hook", "3 Key Points", "Call to Action"],
                "duration": 30
            },
            "trending": {
                "hook": "This {topic} trend is everywhere!",
                "structure": ["Hook", "Why it's trending", "How to join"],
                "duration": 15
            },
            "motivational": {
                "hook": "Here's why {topic} will change your life",
                "structure": ["Hook", "Personal story", "Actionable tip", "Motivation"],
                "duration": 45
            },
            "howto": {
                "hook": "Learn {topic} in 30 seconds",
                "structure": ["Hook", "Step 1", "Step 2", "Step 3", "Result"],
                "duration": 30
            },
            "tips": {
                "hook": "5 {topic} tips you need to know",
                "structure": ["Hook", "Tip 1", "Tip 2", "Tip 3", "Tip 4", "Tip 5"],
                "duration": 60
            }
        }
    
    def initialize_model(self):
        """Initialize the AI model for script generation"""
        try:
            print(f"🔄 Loading {self.model_name}...")
            self.pipe = pipeline("text-generation", model="gpt2")
            print("✅ Model loaded successfully!")
        except Exception as e:
            print(f"❌ Error loading model: {e}")
    
    def generate_script(self, topic, reel_type="educational", target_audience="general", duration=30):
        """Generate a complete reel script based on topic and type"""
        
        template = self.templates.get(reel_type, self.templates["educational"])
        
        # Create the prompt for script generation
        prompt = f"Create an engaging Instagram reel script about {topic} for {target_audience}."
        
        try:
            # Use pipeline for text generation
            result = self.pipe(prompt, max_length=200, num_return_sequences=1, temperature=0.8)
            script = result[0]['generated_text']
            
            # Clean and structure the script
            script = self.clean_script(script)
            
            # Break into scenes
            scenes = self.create_scenes(script, duration)
            
            return {
                "topic": topic,
                "type": reel_type,
                "duration": duration,
                "full_script": script,
                "scenes": scenes,
                "hashtags": self.generate_hashtags(topic),
                "title": self.generate_title(topic, reel_type)
            }
            
        except Exception as e:
            print(f"Error generating script: {e}")
            return self.create_fallback_script(topic, reel_type, duration)
    
    def clean_script(self, script):
        """Clean and format the generated script"""
        script = re.sub(r'\n+', '\n', script)
        script = re.sub(r' +', ' ', script)
        return script.strip()
    
    def create_scenes(self, script, duration):
        """Break script into timed scenes"""
        sentences = re.split(r'[.!?]+', script)
        sentences = [s.strip() for s in sentences if s.strip()]
        
        time_per_scene = duration / len(sentences) if sentences else duration
        
        scenes = []
        current_time = 0
        
        for i, sentence in enumerate(sentences):
            if sentence:
                scene_duration = min(time_per_scene, 8)
                scenes.append({
                    "text": sentence,
                    "start_time": current_time,
                    "duration": scene_duration,
                    "scene_number": i + 1
                })
                current_time += scene_duration
        
        return scenes
    
    def generate_hashtags(self, topic):
        """Generate relevant hashtags"""
        base_hashtags = ["#reels", "#instagram", "#viral", "#trending", "#fyp"]
        topic_words = topic.lower().replace(" ", "").split()
        topic_hashtags = [f"#{word}" for word in topic_words if len(word) > 3]
        
        return base_hashtags + topic_hashtags[:5]
    
    def generate_title(self, topic, reel_type):
        """Generate an engaging title"""
        title_templates = {
            "educational": f"Everything you need to know about {topic}",
            "trending": f"{topic} is trending for a reason!",
            "motivational": f"Why {topic} changed my life",
            "howto": f"How to master {topic} quickly",
            "tips": f"Essential {topic} tips"
        }
        
        return title_templates.get(reel_type, f"Amazing {topic} content!")
    
    def create_fallback_script(self, topic, reel_type, duration):
        """Create a basic script when AI generation fails"""
        fallback_scripts = {
            "educational": f"Let's learn about {topic}! Here are 3 key things you should know. First, {topic} is important because it affects our daily lives. Second, understanding {topic} can help you make better decisions. Third, {topic} is easier to learn than you think. Start your {topic} journey today!",
            "trending": f"Have you seen the {topic} trend? Everyone's talking about it! Here's why it's so popular and how you can join in. Don't miss out on this amazing {topic} trend!",
            "motivational": f"Here's why {topic} will change your life. I discovered {topic} and it transformed everything. You can do this too! Start with small steps and believe in yourself. Your {topic} journey starts now!"
        }
        
        script = fallback_scripts.get(reel_type, f"Amazing content about {topic}! Let's explore this topic together and learn something new.")
        
        return {
            "topic": topic,
            "type": reel_type,
            "duration": duration,
            "full_script": script,
            "scenes": self.create_scenes(script, duration),
            "hashtags": self.generate_hashtags(topic),
            "title": self.generate_title(topic, reel_type)
        }

# Initialize the script generator
script_gen = ReelsScriptGenerator()

## 2. Text-to-Speech Engine

In [ ]:
class TextToSpeechEngine:
    def __init__(self):
        self.output_dir = Path("generated_reels/audio")
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
        self.voices = {
            "english_us": "en",
            "english_uk": "en-uk",
            "spanish": "es",
            "french": "fr",
            "german": "de",
            "italian": "it"
        }
    
    def generate_voiceover(self, text, voice="english_us", speed=1.0):
        """Generate voiceover audio from text"""
        try:
            lang = self.voices.get(voice, "en")
            
            tts = gTTS(text=text, lang=lang, slow=False if speed >= 1.0 else True)
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            audio_file = self.output_dir / f"voiceover_{timestamp}.mp3"
            
            tts.save(str(audio_file))
            
            if speed != 1.0:
                audio_file = self.adjust_speed(audio_file, speed)
            
            print(f"✅ Voiceover generated: {audio_file}")
            return str(audio_file)
            
        except Exception as e:
            print(f"❌ Error generating voiceover: {e}")
            return None
    
    def adjust_speed(self, audio_file, speed):
        """Adjust playback speed of audio"""
        try:
            audio = AudioSegment.from_mp3(audio_file)
            
            faster_audio = audio._spawn(audio.raw_data, overrides={
                "frame_rate": int(audio.frame_rate * speed)
            }).set_frame_rate(audio.frame_rate)
            
            adjusted_file = str(audio_file).replace(".mp3", f"_speed_{speed}.mp3")
            faster_audio.export(adjusted_file, format="mp3")
            
            return adjusted_file
            
        except Exception as e:
            print(f"❌ Error adjusting speed: {e}")
            return audio_file
    
    def generate_scene_audio(self, scenes, voice="english_us", speed=1.0):
        """Generate audio for each scene"""
        audio_files = []
        
        for i, scene in enumerate(scenes):
            text = scene["text"]
            audio_file = self.generate_voiceover(text, voice, speed)
            
            if audio_file:
                audio_files.append({
                    "scene_number": scene["scene_number"],
                    "audio_file": audio_file,
                    "text": text,
                    "duration": scene["duration"]
                })
        
        return audio_files

# Initialize TTS engine
tts_engine = TextToSpeechEngine()

## 3. Video Generation Engine

In [ ]:
class VideoGenerator:
    def __init__(self):
        self.output_dir = Path("generated_reels/videos")
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
        # Video settings for Instagram Reels
        self.width = 1080
        self.height = 1920  # 9:16 aspect ratio
        self.fps = 30
        
        # Color schemes
        self.color_schemes = {
            "vibrant": [(255, 59, 92), (255, 190, 92), (255, 92, 92)],
            "professional": [(45, 52, 69), (90, 103, 127), (170, 170, 170)],
            "nature": [(76, 175, 80), (139, 195, 74), (205, 220, 57)],
            "sunset": [(255, 152, 0), (255, 193, 7), (255, 235, 59)],
            "ocean": [(3, 169, 244), (33, 150, 243), (63, 81, 181)]
        }
    
    def create_text_clip(self, text, duration, style="modern", color_scheme="vibrant"):
        """Create a text clip with animations"""
        try:
            wrapped_text = textwrap.fill(text, width=30)
            
            txt_clip = TextClip(
                wrapped_text,
                fontsize=60,
                color='white',
                font='Arial-Bold',
                align='center',
                method='caption',
                size=(self.width-100, None)
            ).set_duration(duration)
            
            txt_clip = txt_clip.set_position('center')
            
            if style == "fade":
                txt_clip = txt_clip.fadein(0.5).fadeout(0.5)
            elif style == "slide":
                txt_clip = txt_clip.set_position(lambda t: ('center', 50 + t*20))
            elif style == "zoom":
                txt_clip = txt_clip.resize(lambda t: 1 + 0.1*t)
            
            return txt_clip
            
        except Exception as e:
            print(f"❌ Error creating text clip: {e}")
            return None
    
    def create_background(self, duration, style="gradient", color_scheme="vibrant"):
        """Create animated background"""
        try:
            colors = self.color_schemes.get(color_scheme, self.color_schemes["vibrant"])
            
            if style == "solid":
                bg_clip = ColorClip(
                    size=(self.width, self.height),
                    color=colors[0],
                    duration=duration
                )
            
            elif style == "gradient":
                def make_gradient_frame(t):
                    gradient = np.zeros((self.height, self.width, 3), dtype=np.uint8)
                    
                    offset = int(t * 50) % 100
                    
                    for y in range(self.height):
                        ratio = (y + offset) / self.height
                        if ratio > 1:
                            ratio = 2 - ratio
                        
                        color = [
                            int(colors[0][i] * (1-ratio) + colors[1][i] * ratio)
                            for i in range(3)
                        ]
                        gradient[y, :] = color
                    
                    return gradient
                
                bg_clip = VideoClip(make_gradient_frame, duration=duration)
            
            else:
                bg_clip = ColorClip(
                    size=(self.width, self.height),
                    color=colors[0],
                    duration=duration
                )
            
            return bg_clip
            
        except Exception as e:
            print(f"❌ Error creating background: {e}")
            return ColorClip(
                size=(self.width, self.height),
                color=(0, 0, 0),
                duration=duration
            )
    
    def create_scene_video(self, scene, background_style="gradient", text_style="fade", color_scheme="vibrant"):
        """Create video for a single scene"""
        try:
            duration = scene["duration"]
            text = scene["text"]
            
            background = self.create_background(duration, background_style, color_scheme)
            text_clip = self.create_text_clip(text, duration, text_style, color_scheme)
            
            if text_clip is None:
                return background
            
            scene_video = CompositeVideoClip([background, text_clip])
            
            return scene_video
            
        except Exception as e:
            print(f"❌ Error creating scene video: {e}")
            return None
    
    def generate_reel(self, script_data, audio_files=None, background_style="gradient", 
                     text_style="fade", color_scheme="vibrant", background_music=None):
        """Generate complete Instagram reel"""
        try:
            scenes = script_data["scenes"]
            topic = script_data["topic"]
            
            print(f"🎬 Generating reel for: {topic}")
            
            video_clips = []
            audio_clips = []
            
            for i, scene in enumerate(scenes):
                print(f"🎥 Creating scene {i+1}/{len(scenes)}")
                
                scene_video = self.create_scene_video(scene, background_style, text_style, color_scheme)
                
                if scene_video:
                    video_clips.append(scene_video)
                
                if audio_files and i < len(audio_files):
                    try:
                        audio_clip = AudioFileClip(audio_files[i]["audio_file"])
                        audio_clip = audio_clip.set_duration(scene["duration"])
                        audio_clips.append(audio_clip)
                    except Exception as e:
                        print(f"⚠️ Warning: Could not load audio for scene {i+1}: {e}")
            
            if not video_clips:
                raise Exception("No video clips were created")
            
            final_video = concatenate_videoclips(video_clips)
            
            if audio_clips:
                final_audio = concatenate_audioclips(audio_clips)
                final_video = final_video.set_audio(final_audio)
            
            if background_music and os.path.exists(background_music):
                try:
                    bg_music = AudioFileClip(background_music)
                    bg_music = bg_music.volumex(0.3)
                    bg_music = bg_music.set_duration(final_video.duration)
                    
                    if final_video.audio:
                        mixed_audio = CompositeAudioClip([final_video.audio, bg_music])
                        final_video = final_video.set_audio(mixed_audio)
                    else:
                        final_video = final_video.set_audio(bg_music)
                except Exception as e:
                    print(f"⚠️ Warning: Could not add background music: {e}")
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            safe_topic = re.sub(r'[^\w\s-]', '', topic).strip().replace(' ', '_')
            output_file = self.output_dir / f"reel_{safe_topic}_{timestamp}.mp4"
            
            print(f"💾 Exporting video: {output_file}")
            final_video.write_videofile(
                str(output_file),
                fps=self.fps,
                codec='libx264',
                audio_codec='aac',
                temp_audiofile='temp-audio.m4a',
                remove_temp=True
            )
            
            final_video.close()
            
            print(f"✅ Reel generated successfully: {output_file}")
            return str(output_file)
            
        except Exception as e:
            print(f"❌ Error generating reel: {e}")
            return None

# Initialize video generator
video_gen = VideoGenerator()

## 4. Complete Reels Generator

In [ ]:
class InstagramReelsGenerator:
    def __init__(self):
        self.script_gen = ReelsScriptGenerator()
        self.tts_engine = TextToSpeechEngine()
        self.video_gen = VideoGenerator()
        
        self.base_dir = Path("generated_reels")
        self.base_dir.mkdir(exist_ok=True)
        
        (self.base_dir / "scripts").mkdir(exist_ok=True)
        (self.base_dir / "audio").mkdir(exist_ok=True)
        (self.base_dir / "videos").mkdir(exist_ok=True)
    
    def generate_complete_reel(self, topic, reel_type="educational", voice="english_us", 
                              background_style="gradient", color_scheme="vibrant", 
                              duration=30, include_voiceover=True, background_music=None):
        """Generate a complete Instagram reel from topic to video"""
        
        try:
            print(f"🚀 Starting reel generation for: {topic}")
            
            print("📝 Generating script...")
            script_data = self.script_gen.generate_script(topic, reel_type, duration=duration)
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            script_file = self.base_dir / "scripts" / f"script_{topic.replace(' ', '_')}_{timestamp}.json"
            
            with open(script_file, 'w') as f:
                json.dump(script_data, f, indent=2)
            
            print(f"✅ Script generated and saved: {script_file}")
            
            audio_files = None
            if include_voiceover:
                print("🎙️ Generating voiceover...")
                audio_files = self.tts_engine.generate_scene_audio(
                    script_data["scenes"], voice=voice
                )
                print(f"✅ Voiceover generated for {len(audio_files)} scenes")
            
            print("🎬 Generating video...")
            video_file = self.video_gen.generate_reel(
                script_data,
                audio_files=audio_files,
                background_style=background_style,
                color_scheme=color_scheme,
                background_music=background_music
            )
            
            if video_file:
                print(f"🎉 Reel generation completed!")
                
                result = {
                    "success": True,
                    "video_file": video_file,
                    "script_file": str(script_file),
                    "script_data": script_data,
                    "audio_files": audio_files,
                    "settings": {
                        "topic": topic,
                        "type": reel_type,
                        "voice": voice,
                        "background_style": background_style,
                        "color_scheme": color_scheme,
                        "duration": duration
                    }
                }
                
                return result
            else:
                raise Exception("Video generation failed")
                
        except Exception as e:
            print(f"❌ Error in reel generation: {e}")
            return {
                "success": False,
                "error": str(e),
                "script_data": script_data if 'script_data' in locals() else None
            }
    
    def preview_script(self, topic, reel_type="educational", duration=30):
        """Generate and preview just the script"""
        return self.script_gen.generate_script(topic, reel_type, duration=duration)
    
    def get_available_options(self):
        """Get all available options for customization"""
        return {
            "reel_types": list(self.script_gen.templates.keys()),
            "voices": list(self.tts_engine.voices.keys()),
            "background_styles": ["solid", "gradient"],
            "text_styles": ["fade", "slide", "zoom", "modern"],
            "color_schemes": list(self.video_gen.color_schemes.keys())
        }

# Initialize the complete reels generator
reels_generator = InstagramReelsGenerator()

## 5. Gradio Interface

In [ ]:
def create_reels_interface():
    """Create comprehensive Gradio interface for reel generation"""
    
    def generate_script_preview(topic, reel_type, duration):
        """Generate script preview"""
        if not topic.strip():
            return "Please enter a topic", "", "", ""
        
        try:
            script_data = reels_generator.preview_script(topic, reel_type, duration)
            
            script_text = script_data["full_script"]
            title = script_data["title"]
            hashtags = " ".join(script_data["hashtags"])
            
            scenes_text = "\n".join([
                f"Scene {scene['scene_number']}: {scene['text']} ({scene['duration']:.1f}s)"
                for scene in script_data["scenes"]
            ])
            
            return script_text, title, hashtags, scenes_text
            
        except Exception as e:
            return f"Error: {str(e)}", "", "", ""
    
    def generate_full_reel(topic, reel_type, voice, background_style, color_scheme, 
                          duration, include_voiceover, progress=gr.Progress()):
        """Generate complete reel with progress tracking"""
        if not topic.strip():
            return "Please enter a topic", None
        
        try:
            progress(0.1, desc="Initializing...")
            
            progress(0.2, desc="Generating script...")
            result = reels_generator.generate_complete_reel(
                topic=topic,
                reel_type=reel_type,
                voice=voice,
                background_style=background_style,
                color_scheme=color_scheme,
                duration=duration,
                include_voiceover=include_voiceover
            )
            
            progress(1.0, desc="Complete!")
            
            if result["success"]:
                return f"✅ Reel generated successfully!\nVideo: {result['video_file']}", result["video_file"]
            else:
                return f"❌ Error: {result['error']}", None
                
        except Exception as e:
            return f"❌ Error: {str(e)}", None
    
    # Get available options
    options = reels_generator.get_available_options()
    
    # Custom CSS
    css = """
    .gradio-container {
        font-family: 'Arial', sans-serif;
    }
    .generate-btn {
        background: linear-gradient(45deg, #ff6b6b, #4ecdc4);
        border: none;
        color: white;
        font-weight: bold;
    }
    """
    
    with gr.Blocks(css=css, title="Instagram Reels Generator", theme=gr.themes.Soft()) as demo:
        gr.Markdown(
            """
            # 🎬 Instagram Reels Generator
            
            Create engaging Instagram reels with AI-powered scripts and automated video generation!
            
            **Features:**
            - 🤖 AI script generation
            - 🎙️ Text-to-speech voiceovers
            - 🎨 Customizable visual styles
            - 📱 Instagram-optimized format (9:16)
            """
        )
        
        with gr.Tabs():
            # Script Preview Tab
            with gr.TabItem("📝 Script Preview"):
                with gr.Row():
                    with gr.Column(scale=1):
                        topic_input = gr.Textbox(
                            label="Topic",
                            placeholder="e.g., productivity tips, cooking hacks, fitness motivation",
                            lines=2
                        )
                        
                        reel_type_select = gr.Dropdown(
                            choices=options["reel_types"],
                            value="educational",
                            label="Reel Type"
                        )
                        
                        duration_slider = gr.Slider(
                            minimum=15,
                            maximum=90,
                            value=30,
                            step=5,
                            label="Duration (seconds)"
                        )
                        
                        preview_btn = gr.Button("Generate Script Preview", variant="primary")
                    
                    with gr.Column(scale=2):
                        script_output = gr.Textbox(
                            label="Generated Script",
                            lines=10,
                            interactive=False
                        )
                        
                        title_output = gr.Textbox(
                            label="Title",
                            interactive=False
                        )
                        
                        hashtags_output = gr.Textbox(
                            label="Hashtags",
                            interactive=False
                        )
                        
                        scenes_output = gr.Textbox(
                            label="Scene Breakdown",
                            lines=8,
                            interactive=False
                        )
                
                preview_btn.click(
                    generate_script_preview,
                    inputs=[topic_input, reel_type_select, duration_slider],
                    outputs=[script_output, title_output, hashtags_output, scenes_output]
                )
                
                gr.Examples(
                    examples=[
                        ["productivity tips", "tips", 45],
                        ["morning routine", "motivational", 30],
                        ["cooking pasta", "howto", 30],
                        ["AI technology", "educational", 60],
                        ["workout motivation", "motivational", 30]
                    ],
                    inputs=[topic_input, reel_type_select, duration_slider]
                )
            
            # Full Generation Tab
            with gr.TabItem("🎬 Generate Reel"):
                with gr.Row():
                    with gr.Column(scale=1):
                        gr.Markdown("### Content Settings")
                        
                        full_topic_input = gr.Textbox(
                            label="Topic",
                            placeholder="Enter your reel topic",
                            lines=2
                        )
                        
                        full_reel_type = gr.Dropdown(
                            choices=options["reel_types"],
                            value="educational",
                            label="Reel Type"
                        )
                        
                        full_duration = gr.Slider(
                            minimum=15,
                            maximum=90,
                            value=30,
                            step=5,
                            label="Duration (seconds)"
                        )
                        
                        gr.Markdown("### Audio Settings")
                        
                        include_voiceover = gr.Checkbox(
                            value=True,
                            label="Include Voiceover"
                        )
                        
                        voice_select = gr.Dropdown(
                            choices=options["voices"],
                            value="english_us",
                            label="Voice Language"
                        )
                        
                        gr.Markdown("### Visual Settings")
                        
                        background_style = gr.Dropdown(
                            choices=options["background_styles"],
                            value="gradient",
                            label="Background Style"
                        )
                        
                        color_scheme = gr.Dropdown(
                            choices=options["color_schemes"],
                            value="vibrant",
                            label="Color Scheme"
                        )
                        
                        generate_btn = gr.Button(
                            "🎬 Generate Complete Reel", 
                            variant="primary",
                            elem_classes=["generate-btn"]
                        )
                    
                    with gr.Column(scale=2):
                        status_output = gr.Textbox(
                            label="Generation Status",
                            lines=5,
                            interactive=False
                        )
                        
                        video_output = gr.Video(
                            label="Generated Reel",
                            height=600
                        )
                        
                        gr.Markdown(
                            """
                            **Note:** Video generation may take 2-5 minutes depending on duration and settings.
                            Generated files are saved in the `generated_reels` folder.
                            """
                        )
                
                generate_btn.click(
                    generate_full_reel,
                    inputs=[
                        full_topic_input, full_reel_type, voice_select,
                        background_style, color_scheme, full_duration, include_voiceover
                    ],
                    outputs=[status_output, video_output]
                )
            
            # Settings Info Tab
            with gr.TabItem("⚙️ Settings Guide"):
                gr.Markdown(
                    """
                    ## 📋 Reel Types
                    
                    - **Educational**: Informative content with key points
                    - **Trending**: Current trend-focused content
                    - **Motivational**: Inspirational and motivating content
                    - **How-to**: Step-by-step instructional content
                    - **Tips**: Quick tips and advice format
                    
                    ## 🎨 Visual Styles
                    
                    **Background Styles:**
                    - **Solid**: Single color background
                    - **Gradient**: Animated gradient background
                    
                    **Color Schemes:**
                    - **Vibrant**: Bright, energetic colors
                    - **Professional**: Clean, business-like colors
                    - **Nature**: Green, natural tones
                    - **Sunset**: Warm orange and yellow tones
                    - **Ocean**: Cool blue tones
                    
                    ## 🎙️ Voice Options
                    
                    - **English (US)**: American English accent
                    - **English (UK)**: British English accent
                    - **Spanish**: Spanish language
                    - **French**: French language
                    - **German**: German language
                    - **Italian**: Italian language
                    
                    ## 💡 Tips for Best Results
                    
                    1. **Be Specific**: Use detailed topics for better script generation
                    2. **Match Type to Content**: Choose the reel type that fits your message
                    3. **Optimal Duration**: 30-60 seconds work best for engagement
                    4. **Test Different Styles**: Try various color schemes and backgrounds
                    5. **Preview First**: Always check the script before generating video
                    """
                )
        
        gr.Markdown(
            """
            ---
            **Instagram Reels Generator** - Create viral content with AI assistance!
            
            Generated videos are optimized for Instagram Reels (9:16 aspect ratio, 30fps)
            """
        )
    
    return demo

# Create and launch the interface
reels_interface = create_reels_interface()

In [ ]:
# Launch the Gradio interface
# reels_interface.launch(share=False, debug=True)

## 6. Test the Generator

In [ ]:
# Test script generation
test_topic = "productivity tips"
print(f"🧪 Testing script generation for: {test_topic}")

script_preview = reels_generator.preview_script(
    topic=test_topic,
    reel_type="tips",
    duration=45
)

print(f"\n📋 Generated Script:")
print(f"Topic: {script_preview['topic']}")
print(f"Type: {script_preview['type']}")
print(f"Title: {script_preview['title']}")
print(f"\nFull Script:\n{script_preview['full_script']}")
print(f"\nHashtags: {' '.join(script_preview['hashtags'])}")
print(f"\nScenes ({len(script_preview['scenes'])}):{{}")
for i, scene in enumerate(script_preview['scenes']):
    print(f"  {i+1}. [{scene['duration']:.1f}s] {scene['text']}")

In [ ]:
# Test complete reel generation (commented out to avoid resource usage)
# Uncomment to test full video generation

# test_result = reels_generator.generate_complete_reel(
#     topic="morning routine",
#     reel_type="motivational",
#     voice="english_us",
#     background_style="gradient",
#     color_scheme="sunrise",
#     duration=30,
#     include_voiceover=True
# )

# if test_result["success"]:
#     print(f"🎉 Test successful!")
#     print(f"Video file: {test_result['video_file']}")
#     print(f"Script file: {test_result['script_file']}")
# else:
#     print(f"❌ Test failed: {test_result['error']}")

print("📋 Available options:")
options = reels_generator.get_available_options()
for key, values in options.items():
    print(f"  {key}: {values}")

## Summary

This Instagram Reels Generator provides a complete solution for creating engaging short videos:

### Core Features:
1. **AI Script Generation**: Creates engaging scripts based on topic and style
2. **Text-to-Speech**: Generates voiceovers in multiple languages
3. **Video Creation**: Produces Instagram-optimized videos (9:16 ratio)
4. **Customization**: Multiple visual styles and color schemes
5. **Gradio Interface**: User-friendly web interface

### Reel Types:
- **Educational**: Informative content with key points
- **Trending**: Current trend-focused content
- **Motivational**: Inspirational messaging
- **How-to**: Step-by-step instructions
- **Tips**: Quick advice format

### Usage:
1. **Preview Scripts**: Test different topics and styles
2. **Generate Complete Reels**: Create full videos with audio
3. **Customize Visuals**: Choose colors, backgrounds, and animations
4. **Multi-language Support**: Generate voiceovers in various languages

**Output**: Instagram-ready MP4 videos with proper aspect ratio, text overlays, voiceovers, and engaging visuals.

To use: Uncomment the launch command and run the Gradio interface for interactive reel generation!